## Simple JSON Q&A Approach (LangChain)

When we have a question / answer dataset with relatively short answers, we don't need to use a splitter



In [ ]:
import json
# Load JSON data
with open("../data/home0001qa.json", "r") as file:
    qa_data = json.load(file)

In [ ]:
from langchain.schema import Document
# Prepare documents for LangChain
documents = [
    Document(page_content=item["answer"], metadata={"question": item["question"]})
    for item in qa_data
]

documents[:5]

In [ ]:
# turn into function
def prepare_qa_documents(file_path):
    with open(file_path, 'r') as f:
        qa_data = json.load(f)
    
    documents = [
        Document(
            page_content=item["answer"],
            metadata={"question": item["question"]}
        )
        for item in qa_data
    ]
    
    return documents

print(prepare_qa_documents("../data/home0001qa.json")[:5])

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def setup_rag(documents):
    
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(documents, embeddings)
    
    llm = ChatOpenAI(model="gpt-4o-mini")
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", """Use the following similar Q&A pairs to help answer the question. 
        If the context is relevant, use it to answer. If not, say you don't have enough information.
        
        Context Q&A pairs:
        {context}
        """),
        ("human", "{question}")
    ])
    
    chain = (
        {"context": vectorstore.as_retriever(search_type="similarity", k=3), "question": RunnablePassthrough()}
        | rag_prompt
        | llm
        | StrOutputParser()
    )
    
    return chain

In [ ]:
docs = prepare_qa_documents('../data/home0001qa.json')
chain = setup_rag(docs)
print(chain.invoke("Do i own my 0001 home outright?"))

## Basic RAG pipeline w/ Memory (LangGraph)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)
document_retriever = vectorstore.as_retriever()

# System prompt that explains how to handle questions with chat history
question_context_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
# Create prompt template for contextualizing questions
question_contextualization_template = ChatPromptTemplate.from_messages(
    [
        ("system", question_context_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, 
    document_retriever, 
    question_contextualization_template
)

### Answer question ###
answer_generation_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
answer_generation_template = ChatPromptTemplate.from_messages(
    [
        ("system", answer_generation_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create chain for generating answers from documents
question_answer_chain = create_stuff_documents_chain(llm, answer_generation_template)

# Create the complete RAG chain with memory
rag_pipeline_with_memory = create_retrieval_chain(
    history_aware_retriever, 
    question_answer_chain
)

User Question  
↓  
Add Chat History Context  
↓  
Find Relevant Documents  
↓  
Generate Answer  
↓  
Return Response

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from typing import Sequence
from typing_extensions import Annotated, TypedDict

### Statefully manage chat history ###
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


def call_model(state: State):
    response = rag_pipeline_with_memory.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

result = app.invoke(
    {"input": "hi my name is Flippy, what is home001?"},
    config=config,
)
print(result["answer"])

Idea: Update Chat Hoistory w/ Operator corrected messages?

In [ ]:
chat_history = app.get_state(config).values["chat_history"]
for message in chat_history:
    message.pretty_print()

## Manual Document Grader
https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_adaptive_rag_local/

In [ ]:
# Doc grader instructions
doc_grader_instructions = """You are a grader assessing relevance of a retrieved document to a user question.

If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant."""

# Grader prompt
doc_grader_prompt = """Here is the retrieved document: \n\n {document} \n\n Here is the user question: \n\n {question}. 

This carefully and objectively assess whether the document contains at least some information that is relevant to the question.

Return JSON with single key, binary_score, that is 'yes' or 'no' score to indicate whether the document contains at least some information that is relevant to the question."""

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama

llm_model = "llama3.2:3b-instruct-fp16"

llm_json_mode = ChatOllama(model=llm_model, temperature=0, format='json')

from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents, embeddings)

retriever = vectorstore.as_retriever(k=3)

# Test
question = "Does furniture come included?"
docs = retriever.invoke(question)
doc_txt = docs[0].page_content
doc_grader_prompt_formatted = doc_grader_prompt.format(
    document=doc_txt, question=question
)
result = llm_json_mode.invoke(
    [SystemMessage(content=doc_grader_instructions)]
    + [HumanMessage(content=doc_grader_prompt_formatted)]
)

print(doc_txt)
print(json.loads(result.content))

In [ ]:
def grade_documents(documents, question):
  
    # Score each doc
    filtered_docs = []

    for d in documents:
        doc_grader_prompt_formatted = doc_grader_prompt.format(
            document=d.page_content, question=question
        )
        result = llm_json_mode.invoke(
            [SystemMessage(content=doc_grader_instructions)]
            + [HumanMessage(content=doc_grader_prompt_formatted)]
        )
        grade = json.loads(result.content)["binary_score"]
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            continue

    return {"documents": filtered_docs}

In [ ]:
question = "Do i own my 0001 home outright?"
retrieved_docs = retriever.invoke(question)
filtered_docs = grade_documents(retrieved_docs, question)

In [ ]:
# TO DO: 
# ANSWER GRADER 